# Gender-and-Age-Detection

**To build a gender and age detector that can approximately guess the gender and age of the person (face) in a picture or through webcam**

This project can be divided into four parts:

* Detect Faces
* Detect Gender
* Detect Age
* Display output

**Step-1 Face detection function** 

This model was included in OpenCV from version 3.3. It is based on **Single-Shot-Multibox detector** and uses ResNet-10 Architecture as backbone. The model was trained using images available from the web.

In [1]:
# importing the libraries
import numpy as np
import cv2
import math



Function for face detection

In [2]:
def face(net,frame,confidence_threshold=0.7):
    frame_copy=frame.copy()
    image_width=frame_copy.shape[1]
    image_height=frame_copy.shape[0]
    # now we have to take blog from images
    blob=cv2.dnn.blobFromImages(frame_copy,1.0, (300, 300), [104, 117, 123], True, False)
    
    # now to apply our neural network to images as one forward pass
    net.setInput(blob)
    detections=net.forward()
    faceboxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>confidence_threshold:
            # dimensions for drawing the rectangle
            x1=int(detections[0,0,i,3]*image_width)
            y1=int(detections[0,0,i,4]*image_height)
            x2=int(detections[0,0,i,5]*image_width)
            y2=int(detections[0,0,i,6]*image_height)
            faceboxes.append([x1,y1,x2,y2])
            cv2.rectangle(frame_copy, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frame_copy,faceboxes            


def highlightFace(net, frame, conf_threshold=0.7):
    frameOpencvDnn=frame.copy()
    frameHeight=frameOpencvDnn.shape[0]
    frameWidth=frameOpencvDnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections=net.forward()
    print(detections)
    faceBoxes=[]
    for i in range(detections.shape[2]):
        confidence=detections[0,0,i,2]
        if confidence>conf_threshold:
            x1=int(detections[0,0,i,3]*frameWidth)
            y1=int(detections[0,0,i,4]*frameHeight)
            x2=int(detections[0,0,i,5]*frameWidth)
            y2=int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencvDnn, (x1,y1), (x2,y2), (0,255,0), int(round(frameHeight/150)), 8)
    return frameOpencvDnn,faceBoxes



**Step-2 Calling the deep learning models**

In [3]:
faceProto="opencv_face_detector.pbtxt"
faceModel="opencv_face_detector_uint8.pb"
ageProto="age_deploy.prototxt"
ageModel="age_net.caffemodel"
genderProto="gender_deploy.prototxt"
genderModel="gender_net.caffemodel"

*Classes for the gender and age predictions*

In [4]:
ageList=['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(21-32)', '(38-43)', '(48-53)', '(60-100)']
genderList=['Male','Female']

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto) 


*Opening the video or camera for prediction*

In [6]:
video=cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    rect,frame=video.read()
    # taking the frames with rectangle and rectangle size
    resultImg,faceBoxes=highlightFace(faceNet,frame)
    for faceBox in faceBoxes:
        
        print(len(faceBox))
        blob=cv2.dnn.blobFromImage(frame, 1.0, (227,227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        # PREDICTING THE GENDER IN THE IMAGE BY PASSING OUR NEURAL NETWORK ONCE EVEN TO PREDICT MANY PERSONS IN SINGLE IMAGES
        genderNet.setInput(blob)
        genderPreds=genderNet.forward()
        #identifing the class by taking the highest probability number
        gender=genderList[genderPreds[0].argmax()]
        print(genderPreds)
        
        #print(f'Gender: {gender}')

        ageNet.setInput(blob)
        agePreds=ageNet.forward()
        #print(agePreds)
        age=ageList[agePreds[0].argmax()]
        print(agePreds[0].argmax())
        len(agePreds[0])
        #print(f'Age: {age[1:-1]} years')

        cv2.putText(resultImg, f'{gender}, {age}', (faceBox[0], faceBox[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
        cv2.imshow("Detecting age and gender", resultImg)

    
    
    key=cv2.waitKey(1)
    if key == ord('q'):
        break
video.release()
cv2.destroyAllWindows()



[[[[0.         1.         0.998059   ... 0.2681752  0.7182035
    0.6475482 ]
   [0.         1.         0.12774338 ... 4.0022798  0.8359939
    4.9840894 ]
   [0.         1.         0.12627271 ... 4.007157   4.840808
    4.9865746 ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
4
[[0.916008   0.08399202]]
4
[[[[0.         1.         0.997589   ... 0.26782542 0.71708417
    0.6471289 ]
   [0.         1.         0.12734523 ... 4.0018115  0.8346981
    4.985826  ]
   [0.         1.         0.12453885 ... 4.007349   4.8394957
    4.987769  ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]]]
4
[[0.7780833 0.2219167]]
4
[[[[0.         1.         0.